In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import GridSearchCV  
from sklearn.preprocessing import Imputer
from sklearn import linear_model
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
import matplotlib.pyplot as plt
# param2 = { 'n_estimators':[1,5,10,50,100,150,350,500,750],'oob_score':('True','False'),'criterion':('gini','entropy')}

clf = RandomForestClassifier(n_estimators=10, n_jobs=-1)
# clf = GridSearchCV(rf,param2)
# param = {'solver' : ('newton-cg', 'lbfgs', 'liblinear', 'sag'), 'C':[1,10,50,100,500,1000,5000,10000,50000,100000,500000,1000000,5000000]}
# clf=GridSearchCV(clfgs,param)

In [2]:
train_set = pd.read_csv("/home/mohitrbhardwaj/Desktop/kaggle/quora/train.csv")
test_set = pd.read_csv("/home/mohitrbhardwaj/Desktop/kaggle/quora/test.csv")

In [3]:
for x in train_set:
    print(x)

id
qid1
qid2
question1
question2
is_duplicate


In [4]:
# print('Total number of question pairs for training: {}'.format(len(train_set)))
# print('Duplicate pairs: {}%'.format(round(train_set['is_duplicate'].mean()*100, 2)))
# qids = pd.Series(train_set['qid1'].tolist() + train_set['qid2'].tolist())
# print('Total number of questions in the training data: {}'.format(len(np.unique(qids))))
# print('Number of questions that appear multiple times: {}'.format(np.sum(qids.value_counts() > 1)))

# plt.figure(figsize=(12, 5))
# plt.hist(qids.value_counts(), bins=50)
# plt.yscale('log', nonposy='clip')
# plt.title('Log-Histogram of question appearance counts')
# plt.xlabel('Number of occurences of question')
# plt.ylabel('Number of questions')
# print()
# plt.show()

In [5]:
# tfidf_vectorizer = TfidfVectorizer()

# var = 0
# for x in test_set:
#     print(x)
# bar = 0
# ja = 0
# for x in test_set['question1']:
#     if(type(test_set['question1'][bar])==str and type(test_set['question2'][bar])==str):
#         print('yes')
#         ja+=1
#     bar+=1
# if():
#     vect1 = tfidf_vectorizer.fit_transform(test_set['question1'].values.astype('U'))
#     vect2 = tfidf_vectorizer.fit_transform(test_set['question1'].values.astype('U')) 

In [8]:
var = 0
counter = 0 
for x in train_set['question1']:
    lis1 = []
    if(pd.isnull(train_set['question1'][var])==False):
        if(pd.isnull(train_set['question2'][var])==True):
            train_set['question2'][var]="w"
    else:
        train_set['question1'][var]="w"
        if(pd.isnull(train_set['question2'][var])==True):
            train_set['question2'][var]="w"
    var+=1
    counter+=1

In [9]:
var = 0
counter = 0 
for x in test_set['question1']:
    lis1 = []
    if(pd.isnull(test_set['question1'][var])==False):
        if(pd.isnull(test_set['question2'][var])==True):
            test_set['question2'][var]="e"
    else:
        test_set['question1'][var]="e"
        if(pd.isnull(test_set['question2'][var])==True):
            test_set['question2'][var]="e"
    var+=1
    counter+=1

/home/mohitrbhardwaj/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/mohitrbhardwaj/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [11]:
temp = pd.DataFrame()
temp = train_set
temp.to_csv("train1.csv",index=False)
temp = test_set
temp.to_csv("test1.csv",index=False)


In [12]:
train_set = pd.read_csv("/home/mohitrbhardwaj/Desktop/kaggle/quora/train1.csv")
test_set = pd.read_csv("/home/mohitrbhardwaj/Desktop/kaggle/quora/test1.csv")

In [ ]:
for x in test_set:
    print(x)

In [ ]:
# print(res[0])
res = tfidf_vectorizer.fit_transform(train_set['question1']+train_set['question2'])
# res = res
res = res 
tar = train_set['is_duplicate']
clf.fit(res,tar)
test_res = tfidf_vectorizer.transform(test_set['question1']+test_set['question2'])
# test_res.shape
# xx = clf.predict(test_res)

In [ ]:
xx = clf.predict_proba(test_res)
# print(train_set[30000:30004])
# print(res[30000:30001])
xx = np.transpose(xx)
# print(xx[1][:10])

In [ ]:
# plt.plot(test_res[0], 'ro')
# plt.axis([0, 6, 0, 20])
# plt.show()
print(clf.score(tfidf_vectorizer.transform(train_set['question1'][30001:40000]+train_set['question2'][30001:40000]),np.asarray(train_set['is_duplicate'][30001:40000])))

In [ ]:
sol = pd.DataFrame()
sol['test_id']=test_set['test_id']
print("yaha tk ho? gaya")
sol['is_duplicate']=xx[1]
print("yeh bhi")
sol.to_csv("submission.csv",index=False)

In [ ]:
print(sol.head())
print(test_set['question1'].head())
# print(test_set['question2'][:5])

In [ ]:

# print(train_set.head())

# cluster = []    
# k=0
# length=0
# fl = []
# for i in range(1000000):
#     fl.append(False)
# for x in train_set['question1']:
#     if(train_set['is_duplicate'][length]==1):
#         if all([fl[train_set['id1'][length]]==False , fl[train_set['id2'][length]]==False]):
#             cat[train_set['id1'][length]]=k
#             cat[train_set['id2'][length]]=k
#             fl[train_set['id1'][length]]=True
#             fl[train_set['id2'][length]]=True
#             k+=1
#         elif(fl[train_set['id1'][length]]==False):
#             cat[train_set['id1'][length]]=cat[train_set['id2'][length]]
#             fl[train_set['id1'][length]]=True
#         elif(fl[train_set['id2'][length]]==False):
#             cat[train_set['id2'][length]]=cat[train_set['id1'][length]]
#             fl[train_set['id2'][length]]=True
#     else:
#         if(fl[train_set['id1'][length]]==False):
#             cat[train_set['id1'][length]]=k
#             fl[train_set['id1'][length]]=True
#         if(fl[train_set['id2'][length]]==False):
#             cat[train_set['id1'][length]]=k
#             fl[train_set['id1'][length]]=True
            
# for i in range(k+2):
#     cluster.append([])    